# LangGraph Workflows and State Machines - Interactive Notebook

This notebook provides hands-on examples to accompany the lesson in [`09_langgraph_workflows.md`](09_langgraph_workflows.md):
- Define state, nodes, and edges
- Conditional routing and loops
- Streaming execution events
- Persistence with a checkpointer
- LLM/tool integration inside nodes
- Error handling and retries

Ensure your `.env` has OPENAI_API_KEY for LLM examples.

## 0) Setup and Imports

In [ ]:
import os
from typing import TypedDict, Optional, List, Dict, Any

from dotenv import load_dotenv
load_dotenv()

# LangGraph core
from langgraph.graph import StateGraph, START, END

# Optional: LLM for node integration
try:
    from langchain_openai import ChatOpenAI
    from langchain_core.prompts import ChatPromptTemplate
    llm_available = True
except Exception as e:
    print("LLM integrations not available:", e)
    llm_available = False

print("Setup complete.")

## 1) Minimal Linear Flow: State, Nodes, Edges

In [ ]:
class AppState(TypedDict, total=False):
    input_text: str
    processed: str

# Node: normalize input
def normalize(state: AppState) -> AppState:
    text = state.get("input_text", "")
    return {"processed": text.strip().lower()}

# Node: finalize (pass-through)
def finalize(state: AppState) -> AppState:
    return state

# Build and compile graph
builder = StateGraph(AppState)
builder.add_node("normalize", normalize)
builder.add_node("finalize", finalize)
builder.add_edge(START, "normalize")
builder.add_edge("normalize", "finalize")
builder.add_edge("finalize", END)
linear_graph = builder.compile()

# Invoke
out = linear_graph.invoke({"input_text": "  Hello LangGraph  "})
print("Linear output:", out)

## 2) Streaming Execution Events

In [ ]:
print("Streaming events:")
for event in linear_graph.astream_events({"input_text": "  Hi  "}):
    print(event.get("event"), event.get("name"), event.get("timestamp"))

## 3) Conditional Routing

In [ ]:
def is_long(state: AppState) -> str:
    text = state.get("input_text", "")
    return "long" if len(text) > 20 else "short"

def handle_long(state: AppState) -> AppState:
    return {"processed": f"LONG::{state.get('input_text','')}"}

def handle_short(state: AppState) -> AppState:
    return {"processed": f"SHORT::{state.get('input_text','')}"}

router = StateGraph(AppState)
router.add_node("handle_long", handle_long)
router.add_node("handle_short", handle_short)
router.add_conditional_edges(START, is_long, {"long": "handle_long", "short": "handle_short"})
router.add_edge("handle_long", END)
router.add_edge("handle_short", END)
conditional_graph = router.compile()

print(conditional_graph.invoke({"input_text": "tiny"}))
print(conditional_graph.invoke({"input_text": "this string is definitely long"}))

## 4) Loops with a Counter in State

In [ ]:
class LoopState(TypedDict, total=False):
    step: int
    log: List[str]

def stepper(state: LoopState) -> LoopState:
    n = state.get("step", 0) + 1
    log = (state.get("log") or []) + [f"step {n}"]
    return {"step": n, "log": log}

loop_builder = StateGraph(LoopState)
loop_builder.add_node("stepper", stepper)
loop_builder.add_edge(START, "stepper")

# Router to loop until step < 3
def check(state: LoopState) -> str:
    return "again" if state.get("step", 0) < 3 else "done"

loop_builder.add_conditional_edges("stepper", check, {"again": "stepper", "done": END})
loop_graph = loop_builder.compile()

print(loop_graph.invoke({}))

## 5) Integrate LLM Calls Inside Nodes (Optional)

In [ ]:
llm_graph = None
if llm_available:
    try:
        llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
        def summarize(state: AppState) -> AppState:
            text = state.get("input_text", "")
            if not text:
                return {}
            prompt = ChatPromptTemplate.from_messages([
                ("system", "Summarize concisely"),
                ("human", "{text}")
            ])
            messages = prompt.format_messages(text=text)
            resp = llm.invoke(messages)
            return {"processed": resp.content}
        llm_builder = StateGraph(AppState)
        llm_builder.add_node("summarize", summarize)
        llm_builder.add_edge(START, "summarize")
        llm_builder.add_edge("summarize", END)
        llm_graph = llm_builder.compile()
        print(llm_graph.invoke({"input_text": "LangGraph builds stateful LLM workflows."}))
    except Exception as e:
        print("LLM node failed:", e)
else:
    print("LLM unavailable; skipping summarize node demo.")

## 6) Persistence with a Checkpointer (SQLite)

In [ ]:
try:
    from langgraph.checkpoint.sqlite import SqliteSaver
    checkpointer = SqliteSaver.from_conn_string("sqlite:///langgraph_state.db")

    # Reuse the linear builder for persistence demo
    persistent_graph = builder.compile(checkpointer=checkpointer)

    config = {"configurable": {"thread_id": "user-123"}}
    print("Persist invoke 1:", persistent_graph.invoke({"input_text": "hello"}, config=config))
    print("Persist invoke 2:", persistent_graph.invoke({"input_text": "continue"}, config=config))
except Exception as e:
    print("Checkpointer unavailable:", e)

## 7) Error Handling and Conditional Error Routing

In [ ]:
def robust(state: AppState) -> AppState:
    try:
        assert "input_text" in state, "missing input_text"
        return {"processed": state["input_text"].upper()}
    except Exception as e:
        return {"error": str(e)}

def route_errors(state: AppState) -> str:
    return "err" if "error" in state else "ok"

err_builder = StateGraph(AppState)
err_builder.add_node("robust", robust)
err_builder.add_node("on_ok", lambda s: s)
err_builder.add_node("on_err", lambda s: s)
err_builder.add_edge(START, "robust")
err_builder.add_conditional_edges("robust", route_errors, {"ok": "on_ok", "err": "on_err"})
err_builder.add_edge("on_ok", END)
err_builder.add_edge("on_err", END)
err_graph = err_builder.compile()

print("Error path (missing input):", err_graph.invoke({}))
print("OK path:", err_graph.invoke({"input_text": "ok"}))

## 8) Streaming Observability (Async Skeleton)

In [ ]:
import asyncio

async def run_streaming_demo():
    async for ev in linear_graph.astream_events({"input_text": "trace me"}):
        print(ev.get("event"), ev.get("name"), ev.get("timestamp"))

# Uncomment to run async streaming
# asyncio.run(run_streaming_demo())

## 9) Capstone Skeleton: Branching Summarize-or-Classify with Loop

In [ ]:
class PipelineState(TypedDict, total=False):
    text: str
    mode: str
    summary: str
    label: str
    confirm: bool

# Router: choose summarize vs classify based on word count
def choose(state: PipelineState) -> str:
    words = len((state.get("text") or "").split())
    return "summarize" if words > 12 else "classify"

# Classify node (simple heuristic)
def classify(state: PipelineState) -> PipelineState:
    txt = (state.get("text") or "").lower()
    label = "tech" if ("langgraph" in txt or "llm" in txt) else "other"
    return {"label": label}

# Summarize node (stub; integrate LLM if available)
def summarize_node(state: PipelineState) -> PipelineState:
    t = (state.get("text") or "")
    if llm_available:
        try:
            llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
            prompt = ChatPromptTemplate.from_messages([
                ("system", "Summarize succinctly"),
                ("human", "{text}")
            ])
            messages = prompt.format_messages(text=t)
            resp = llm.invoke(messages)
            return {"summary": resp.content}
        except Exception as e:
            return {"summary": f"Failed to summarize: {e}"}
    return {"summary": (t[:50] + "...") if t else ""}

# Loop router: continue until confirm=True
def should_continue(state: PipelineState) -> str:
    return "again" if not state.get("confirm") else "done"

b = StateGraph(PipelineState)
b.add_node("classify", classify)
b.add_node("summarize", summarize_node)
b.add_conditional_edges(START, choose, {"summarize": "summarize", "classify": "classify"})
b.add_conditional_edges("classify", should_continue, {"again": "classify", "done": END})
b.add_conditional_edges("summarize", should_continue, {"again": "summarize", "done": END})
capstone = b.compile()

print(capstone.invoke({"text": "LangGraph builds stateful LLM apps for production", "confirm": True}))

## Summary
You built LangGraph workflows with nodes/edges, conditional routing, loops, streaming events, persistence, LLM integration, and error handling. Extend these patterns to tool-calling nodes and RAG-based nodes as needed.